In [6]:
import ipywidgets as widgets
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from tensorflow import keras
import os
from main import make_model, train_model, get_week, normalize, percentages, binary_increase
from math import log
import datetime
from IPython.display import display
%matplotlib widget

In [2]:
normal_model = []
percentage_model = []
binary_model = []

if os.path.exists('normal_model.keras'):
    normal_model = keras.models.load_model('normal_model.keras')
else:
    normal_model = make_model('linear')
    train_model(normal_model, 'normal')
    normal_model.save('normal_model.keras')
    
if os.path.exists('percentage_model.keras'):
    percentage_model = keras.models.load_model('percentage_model.keras')
else:
    percentage_model = make_model('linear')
    train_model(percentage_model, 'percentage')
    percentage_model.save('percentage_model.keras')
    
if os.path.exists('binary_model.keras'):
    binary_model = keras.models.load_model('binary_model.keras')
else:
    binary_model = make_model('sigmoid')
    train_model(binary_model, 'binary')
    binary_model.save('binary_model.keras')

In [3]:
normal_data = get_week()
for col in normal_data.columns:
    normal_data[col] = normalize(normal_data[col], lambda x:  log(x, 8))
normal_data = normal_data[['tweets', 'price']].to_numpy()

percentage_data = get_week()
for col in percentage_data.columns:
    percentage_data[col] = percentages(percentage_data[col])
percentage_data = percentage_data[['tweets', 'price']].to_numpy()

binary_data = get_week()
for col in binary_data.columns:
    binary_data[col] = binary_increase(binary_data[col])
binary_data = binary_data[['tweets', 'price']].to_numpy()

# Bitcoin Price Forecast

In [34]:
actual_data = get_week()
price_predictor = make_model('linear')
price_predictor.set_weights(normal_model.get_weights())
pred_price = price_predictor.predict(np.array([normal_data,]))
pred_price = 8**pred_price[0][0]
pred_df = pd.DataFrame()
pred_df['date'] = [actual_data.index[-1], actual_data.index[-1] + datetime.timedelta(days=1)]
pred_df['price'] = [actual_data['price'][actual_data.index[-1]], pred_price]
pred_df.set_index('date', inplace=True)
plt.plot(actual_data['price'], label='Actual')

plt.plot(pred_df['price'], label='Predicted')
plt.legend()
plt.xlabel('Date')
plt.ylabel('BTC Price ($)')
label1 = widgets.Label(value='How many days to predict?')
display(label1)
days = widgets.IntSlider(min=1, max=7)
display(days)
compute = widgets.Button(description='Predict')

new_data = actual_data.copy()
new_data['price'] = normalize(new_data['price'], lambda x: log(x, 8))
new_data['tweets'] = normalize(new_data['tweets'], lambda x: log(x, 8))
#new_data.drop(actual_data.index[0], inplace=True)
#new_data.loc[new_data.index[-1] + datetime.timedelta(days=1)] = {'price': log(pred_price, 8), 'tweets': new_data['tweets'][new_data.index[-1]]}

def on_predict(b):
    new_data = actual_data.copy()
    new_data['price'] = normalize(new_data['price'], lambda x: log(x, 8))
    new_data['tweets'] = normalize(new_data['tweets'], lambda x: log(x, 8))
    for i in range(days.value):
        pred_price = price_predictor.predict(np.array([new_data[['tweets', 'price']].to_numpy()]))
        pred_price = 8**pred_price[0][0]
        pred_df.loc[actual_data.index[-1] + datetime.timedelta(days=i+1)] = {'price': pred_price}
        new_data.drop(new_data.index[0], inplace=True)
        new_data.loc[actual_data.index[-1] + datetime.timedelta(days=i+1)] = {'price': log(pred_price, 8), 'tweets': new_data['tweets'][new_data.index[-1]]} 
    plt.plot(pred_df['price'], color='orange')
        
compute.on_click(on_predict)
display(compute)



#pred_df[['price', 'tweets']]

Label(value='How many days to predict?')

IntSlider(value=1, max=7, min=1)

Button(description='Predict', style=ButtonStyle())

In [41]:
binary_label = widgets.HTML(value='<h1>The predicted positive or negative outlook based on the binary model is:</h1>')
display(binary_label)
actual_data = get_week()
binary_predictor = make_model('sigmoid')
binary_predictor.set_weights(binary_model.get_weights())
pred_binary = binary_predictor.predict(np.array([binary_data,]))
pred_binary = 1 if pred_binary[0][0] > .5 else 0
pt = "Positive" if pred_binary==1 else "Negative"
pc = 'green' if pred_binary==1 else 'red'
binary_label1 = widgets.HTML(value = f"<h1><font color={pc}>{pt}</h1>")
display(binary_label1)

Label(value='The predicted positive or negative outlook based on the binary model is:')

HTML(value='<b><font color=green>Positive</b>')

In [44]:
holdings_label = widgets.HTML(value='<h3>Input your current bitcoin holdings in BTC:</h3>')
display(holdings_label)
holdings_input = widgets.FloatText()
display(holdings_input)
holdings_label1 = widgets.HTML(value=f'<h3>After {days.value}</h3>')
holdings_button = widgets.button

HTML(value='<h3>Input your current bitcoin holdings in BTC:</h3>')

FloatText(value=0.0)